# Модерация токсичных комментариев

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Описание-проекта" data-toc-modified-id="Описание-проекта-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Описание проекта</a></span></li><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Подготовка данных</a></span><ul class="toc-item"><li><span><a href="#Изучение-данных" data-toc-modified-id="Изучение-данных-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Изучение данных</a></span></li><li><span><a href="#Обработка-данных" data-toc-modified-id="Обработка-данных-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Обработка данных</a></span></li><li><span><a href="#Подготовка-к-обучению" data-toc-modified-id="Подготовка-к-обучению-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Подготовка к обучению</a></span></li></ul></li><li><span><a href="#Обучение-моделей" data-toc-modified-id="Обучение-моделей-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Обучение моделей</a></span><ul class="toc-item"><li><span><a href="#Подбор-параметров" data-toc-modified-id="Подбор-параметров-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Подбор параметров</a></span></li><li><span><a href="#Тестирование-моделей" data-toc-modified-id="Тестирование-моделей-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Тестирование моделей</a></span></li></ul></li><li><span><a href="#Общие-выводы" data-toc-modified-id="Общие-выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Общие выводы</a></span></li></ul></div>

## Описание проекта

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Нужно обучить модель классифицировать комментарии на позитивные и негативные. Для исследования предоставлен набор данных с разметкой о токсичности правок. Необходимо построить модель со значением метрики качества F1 не меньше 0.75. 

## Подготовка данных

### Изучение данных

In [1]:
# Импортируем необходимые для работы библиотеки.

import matplotlib.pyplot as plt
import nltk
import numpy as np
import pandas as pd
import re
import warnings

from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from nltk.corpus import stopwords as nltk_stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.dummy import DummyClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.utils import shuffle

warnings.filterwarnings('ignore')

In [2]:
# Открываем csv-файл с таблицей и сохраняем в переменную "df_comments".

try:
    df_comments = pd.read_csv('/datasets/toxic_comments.csv', index_col=[0])
except:
    try:
        df_comments = pd.read_csv('datasets/toxic_comments.csv', index_col=[0])
    except:
        print('Проверьте пути к файлам.')

In [3]:
# Просматриваем первые строки датафрейма.

df_comments.head()

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [4]:
# Увеличиваем видимость символов в ячейках до 150.
# Снова просматриваем первые строки датафрейма.

pd.options.display.max_colwidth = 150
df_comments.head()

,text,toxic
0,"Explanation\nWhy the edits made under my username Hardcore Metallica Fan were reverted? They weren't vandalisms, just closure on some GAs after I ...",0
1,"D'aww! He matches this background colour I'm seemingly stuck with. Thanks. (talk) 21:51, January 11, 2016 (UTC)",0
2,"Hey man, I'm really not trying to edit war. It's just that this guy is constantly removing relevant information and talking to me through edits in...",0
3,"""\nMore\nI can't make any real suggestions on improvement - I wondered if the section statistics should be later on, or a subsection of """"types of...",0
4,"You, sir, are my hero. Any chance you remember what page that's on?",0


In [5]:
# Общая информация о датафрейме

df_comments.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159292 entries, 0 to 159450
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159292 non-null  object
 1   toxic   159292 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 3.6+ MB


In [6]:
# Доли уникальных значений в целевом признаке (столбец "toxic")

df_comments['toxic'].value_counts(normalize=True)

0    0.898388
1    0.101612
Name: toxic, dtype: float64

In [7]:
# Находим количественное соотношение классов в целевом признаке.

ratio = (df_comments['toxic'].value_counts(normalize=True)[0] 
         / df_comments['toxic'].value_counts(normalize=True)[1])
print('Соотношение отрицательного класса к положительному:', ratio)

Соотношение отрицательного класса к положительному: 8.841344371679229


In [8]:
print('Количество строк, имеющих дубликаты:', df_comments.duplicated().sum())

Количество строк, имеющих дубликаты: 0


__Выводы:__

1. Таблица представляет собой датасет с 159292 строками и 2 столбцами. Для сокращения времени на обработку желательно уменьшить выборку (без значимого ущерба для качества предсказаний).
 
 
2. В столбце 'text' содержатся тексты сообщений, тип данных - object. В столбце 'toxic' - целевой признак в виде 1 и 0 (соответственно, наличие или отсутствие токсичного комментария), тип данных - int64.


3. Пропущенных значений в таблице нет.


4. Доля нулей в целевом признаке - 89.8%, доля единиц - 10.2%. Строк с нулями примерно в 8.84 раза больше, чем с единицами. Данный дисбаланс классов может повлиять на качество предсказаний, поэтому его необходимо устранить.


5. Полных дубликатов стреди строк не обнаружено. Однако следует перепроверить данный факт после очищения текстов.


6. Перед обучением моделей данные также подлежат очистке, токенизации, лемматизации, выделению признаков и разбивке их на обучающую и тестовую выборки.

### Обработка данных

In [9]:
# Перевод текстов в нижний регистр

df_comments['text'] = df_comments['text'].str.lower()

In [10]:
# Примерное время на обработку - 1 минута

# Очистка
df_comments['text'] = df_comments['text'].apply(
    lambda x: " ".join(re.sub(r"[^a-zA-Z']", " ", x).split()))

# Токенизация
df_comments['text'] = df_comments['text'].apply(lambda x: word_tokenize(x))

# Лемматизация
df_comments['text'] = df_comments['text'].apply(
    lambda x: ' '.join([WordNetLemmatizer().lemmatize(w) for w in x]))

In [11]:
# Проверка на наличие дубликатов

print('Количество строк, имеющих дубликаты:', df_comments.duplicated().sum())

Количество строк, имеющих дубликаты: 1256


In [12]:
# Удаление дубликатов, обновление индексов
# Повторная проверка на наличие дубликатов

df_comments = df_comments.drop_duplicates().reset_index(drop=True)
print('Количество строк, имеющих дубликаты:', df_comments.duplicated().sum())

Количество строк, имеющих дубликаты: 0


In [13]:
# Просматриваем первые строки датафрейма после обработки.

df_comments.head()

,text,toxic
0,explanation why the edits made under my username hardcore metallica fan were reverted they were n't vandalism just closure on some gas after i vot...,0
1,d'aww he match this background colour i 'm seemingly stuck with thanks talk january utc,0
2,hey man i 'm really not trying to edit war it 's just that this guy is constantly removing relevant information and talking to me through edits in...,0
3,more i ca n't make any real suggestion on improvement i wondered if the section statistic should be later on or a subsection of type of accident i...,0
4,you sir are my hero any chance you remember what page that 's on,0


### Подготовка к обучению

In [14]:
# Разбиваем датасет на обучающую и тестовую выборки.
# Проверяем размеры полученных наборов данных.

train, test = train_test_split(df_comments, test_size=0.1, random_state=12345)

print(train.shape)
print(test.shape)

(142232, 2)
(15804, 2)


Для частичного устранения дисбаланса классов и сокращения времени на обработку данных уменьшим количество строк с отрицательным классом в целевом признаке обучающей выборки в `ratio/2` раз.

In [15]:
# Даунсемплинг обучающей выборки

zeros = train[train['toxic'] == 0]
ones = train[train['toxic'] == 1]
train = pd.concat([zeros.sample(frac=2/ratio, random_state=12345), ones])
train = shuffle(train, random_state=12345)

print('Новые размеры обучающей выборки:', train.shape)
print('Новое соотношение классов в целевом признаке:')
train['toxic'].value_counts(normalize=True)

Новые размеры обучающей выборки: (43353, 2)
Новое соотношение классов в целевом признаке:


0    0.666759
1    0.333241
Name: toxic, dtype: float64

In [16]:
# В каждой выборке dыделяем признаки, переводим их в векторный вид.

features_train = train['text'].values
target_train = train['toxic'].values

features_test = test['text'].values
target_test = test['toxic'].values

In [17]:
# Формируем набор стоп-слов.
# Cоздаем счетчик величин TF-IDF.

nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words=stopwords)
count_tf_idf

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\fincr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


TfidfVectorizer(stop_words={'a', 'about', 'above', 'after', 'again', 'against',
                            'ain', 'all', 'am', 'an', 'and', 'any', 'are',
                            'aren', "aren't", 'as', 'at', 'be', 'because',
                            'been', 'before', 'being', 'below', 'between',
                            'both', 'but', 'by', 'can', 'couldn', "couldn't", ...})

In [18]:
# Создаем корпус текстов в трейне и признаки из величин TF-IDF.
# Проверяем размеры полученных наборов данных.

features_train_tfidf = count_tf_idf.fit_transform(features_train)
features_test_tfidf = count_tf_idf.transform(features_test)

print(features_train_tfidf.shape)
print(features_test_tfidf.shape)

(43353, 69589)
(15804, 69589)


__Итоги:__


1. Для экономии времени количество строк датасета уменьшено втрое.


2. Тексты комментариев переведены в нижний регистр.


3. Проведено очищение текстов от лишних символов, кроме латинских букв и апострофа.


4. Проведена токенизация (разбивка текста на отдельные слова).


5. Проведена лемматизация каждого слова в текстах (приведение слова к начальной словарной форме).


6. Удалены полные дубликаты строк.


7. Датасет разбит на обучающую и тестовую выборки.


8. Выполнен даунсемплинг обучающей выборки до размеров, при которых установился баланс между классами в целевом признаке.


9. В каждой выборке выделены признаки и переведены в векторный вид: признак для обучения ('text') и целевой признак ('toxic').


10. Сформирован набор стоп-слов и создан счетчик величин TF-IDF.


11. Создан корпус текстов в обучающей выборке, а также созданы признаки (в обучающей и тестовой выборках) из величин TF-IDF.

## Обучение моделей

### Подбор параметров

Для выбора наиболее оптимальной структуры машинного обучения, подобрав лучшие гиперпараметры, сравним 5 из них (Логистрическую регрессию, Классификатор Ridge, Классификатор случайного леса, Классификатор LGBM и Классификатор CatBoost), а также фиктивный Классификатор Dummy.

In [19]:
def grid_search(model):
    pipe = Pipeline([('tfidf', TfidfVectorizer(stop_words=stopwords)), ('', model)])
    grid = GridSearchCV(pipe, params, scoring='f1', cv=3)
    grid.fit(features_train, target_train)
    print('Лучшие параметры модели:', grid.best_params_)
    print('Метрика F1 на обучающей выборке:', grid.best_score_, '\n')

In [20]:
# LogisticRegression
# Примерное время на обработку - 2 мин

model_lr = LogisticRegression(random_state=12345)
params = [{'__solver':['lbfgs', 'liblinear', 'sag', 'saga'],
           '__C':[0.1, 1, 10],
           '__random_state':[12345]}]
grid_search(model_lr)

Лучшие параметры модели: {'__C': 10, '__random_state': 12345, '__solver': 'sag'}
Метрика F1 на обучающей выборке: 0.8545009767428594 



In [21]:
# RidgeClassifier
# Примерное время на обработку - 1 мин

model_rc = RidgeClassifier(random_state=12345)
params = [{'__solver':['auto', 'svd', 'cholesky', 'lsqr',
                     'sparse_cg', 'sag', 'saga', 'lbfgs'],
           '__random_state':[12345]}]
grid_search(model_rc)

Лучшие параметры модели: {'__random_state': 12345, '__solver': 'sag'}
Метрика F1 на обучающей выборке: 0.8175946643850694 



In [22]:
# RandomForestClassifier
# Примерное время на обработку - 3 мин

model_rfc = RandomForestClassifier(random_state=12345)
params = [{'__n_estimators': [100, 150, 200],
          '__max_depth': range(7, 12),
          '__random_state':[12345],
          '__class_weight':['balanced']}]
grid_search(model_rfc)

Лучшие параметры модели: {'__class_weight': 'balanced', '__max_depth': 11, '__n_estimators': 200, '__random_state': 12345}
Метрика F1 на обучающей выборке: 0.7096354654984434 



In [23]:
# LGBMClassifier
# Примерное время на обработку - 3,5 мин

model_lgbmc = LGBMClassifier(random_state=12345)
params = [{'__max_depth' : [-1], #1
           '__n_estimators' : [500], #100, 300
           '__learning_rate':[0.03], #0.01, 0.1
           '__random_state':[12345]}]
grid_search(model_lgbmc)

Лучшие параметры модели: {'__learning_rate': 0.03, '__max_depth': -1, '__n_estimators': 500, '__random_state': 12345}
Метрика F1 на обучающей выборке: 0.8457151942402582 



In [24]:
# CatBoostClassifier
# Примерное время на обработку - 6 минут

model_cbc = CatBoostClassifier(random_state=12345)
params = [{'__max_depth':[6], #5 
           '__learning_rate':[0.9], #0.3
           '__iterations':[100], #50
           '__verbose':[False],
           '__random_state':[12345]}]
grid_search(model_cbc)

Лучшие параметры модели: {'__iterations': 100, '__learning_rate': 0.9, '__max_depth': 6, '__random_state': 12345, '__verbose': False}
Метрика F1 на обучающей выборке: 0.8387432399564085 



In [25]:
# DummyClassifier

model_dc = DummyClassifier()
params = [{'__strategy':['most_frequent', 'prior', 'stratified', 'uniform', 'constant'], 
           '__random_state':[12345]}]
grid_search(model_dc)

Лучшие параметры модели: {'__random_state': 12345, '__strategy': 'uniform'}
Метрика F1 на обучающей выборке: 0.4017033878674128 



__Выводы:__

1. Меньше всего времени на обработку ушло у модели RidgeClassifier (1 минута), больше всего - у LGBMClassifier (около 30 минут).


2. Наиболее высокий показатель F1-меры на обучающей выборке продемонстрировала модель LogisticRegression (0.8545), наиболее низкий - RandomForestClassifier (0.7012).


3. Поскольку почти у всех моделей успешные результаты, для окончательного выбора необходимо проверить их на тестовой выборке.

### Тестирование моделей

Проверим все 5 моделей с подобранными гиперпараметрами на тестовой выборке. Кроме того вычислим не только F1-меру, но также другие метрики (правильность, точность, полноту), а также выведем данные матрицы ошибок .

In [26]:
''' Функция для проверки моделей на тестовой выборке, 
а также вывода метрик (в т.ч. F1) и данных матрицы ошибок.'''

def model_test(model):
    model.fit(features_train_tfidf, target_train)
    predictions = model.predict(features_test_tfidf)

    accuracy = accuracy_score(target_test, predictions)
    precision = precision_score(target_test, predictions)
    recall = recall_score(target_test, predictions)
    f1 = f1_score(target_test, predictions)
    matrix = confusion_matrix(target_test, predictions)  
    
    print('МЕТРИКИ')
    print('Accuracy:', accuracy)
    print('Precision:', precision)
    print('Recall:', recall)
    print('F1-мера:', f1)
    print('\nМАТРИЦА ОШИБОК')
    print('Истинно-отрицательные ответы:', matrix[0, 0])
    print('Ложно-отрицательные ответы:', matrix[1, 0])
    print('Истинно-положительные ответы:', matrix[1, 1])
    print('Ложно-положительные ответы:', matrix[0, 1])
    print()

In [27]:
# LogisticRegression

model_lr = LogisticRegression(C=10, 
                              solver='lbfgs', 
                              random_state=12345)
model_test(model_lr)

МЕТРИКИ
Accuracy: 0.9455201214882308
Precision: 0.6974439227960355
Recall: 0.8263288009888752
F1-мера: 0.7564356435643564

МАТРИЦА ОШИБОК
Истинно-отрицательные ответы: 13606
Ложно-отрицательные ответы: 281
Истинно-положительные ответы: 1337
Ложно-положительные ответы: 580



In [28]:
# RidgeClassifier

model_rc = RidgeClassifier(solver='auto', 
                           random_state=12345)
model_test(model_rc)

МЕТРИКИ
Accuracy: 0.9417236142748672
Precision: 0.6972269383135258
Recall: 0.761433868974042
F1-мера: 0.727917282127031

МАТРИЦА ОШИБОК
Истинно-отрицательные ответы: 13651
Ложно-отрицательные ответы: 386
Истинно-положительные ответы: 1232
Ложно-положительные ответы: 535



In [29]:
# RandomForestClassifier

model_rfc = RandomForestClassifier(max_depth=11, 
                                   n_estimators=100, 
                                   random_state=12345,
                                   class_weight='balanced')
model_test(model_rfc)

МЕТРИКИ
Accuracy: 0.728802834725386
Precision: 0.25648046732384083
Recall: 0.8683559950556242
F1-мера: 0.3959977452085682

МАТРИЦА ОШИБОК
Истинно-отрицательные ответы: 10113
Ложно-отрицательные ответы: 213
Истинно-положительные ответы: 1405
Ложно-положительные ответы: 4073



In [30]:
# LGBMClassifier
# Время на обработку - около 1 минут

model_lgbmc = LGBMClassifier(learning_rate=0.03, 
                             max_depth=-1, 
                             n_estimators=500, 
                             random_state=12345)
model_test(model_lgbmc)

МЕТРИКИ
Accuracy: 0.9513414325487218
Precision: 0.748390871854886
Recall: 0.7904820766378244
F1-мера: 0.7688608355876165

МАТРИЦА ОШИБОК
Истинно-отрицательные ответы: 13756
Ложно-отрицательные ответы: 339
Истинно-положительные ответы: 1279
Ложно-положительные ответы: 430



In [31]:
# CatBoostClassifier
# Время на обработку - около 1,5 минут

model_cbc = CatBoostClassifier(iterations=100, 
                               learning_rate=0.9, 
                               max_depth=6, 
                               verbose=False, 
                               random_state=12345)
model_test(model_cbc)

МЕТРИКИ
Accuracy: 0.939888635788408
Precision: 0.673777315296566
Recall: 0.8003708281829419
F1-мера: 0.731638418079096

МАТРИЦА ОШИБОК
Истинно-отрицательные ответы: 13559
Ложно-отрицательные ответы: 323
Истинно-положительные ответы: 1295
Ложно-положительные ответы: 627



In [32]:
# DummyClassifier

model_dc = DummyClassifier(strategy='uniform')
model_test(model_dc)

МЕТРИКИ
Accuracy: 0.5062642369020501
Precision: 0.10720182903594563
Recall: 0.5216316440049443
F1-мера: 0.1778527025603203

МАТРИЦА ОШИБОК
Истинно-отрицательные ответы: 7157
Ложно-отрицательные ответы: 774
Истинно-положительные ответы: 844
Ложно-положительные ответы: 7029



__Выводы:__

1. Результаты предсказании на тестовой выборке (значения F1-меры):
    - LogisticRegression - 0.7559;
    - RidgeClassifier - 0.7279;
    - RandomForestClassifier - 0.3694;
    - LGBMClassifier - 0.7691;
    - CatBoostClassifier - 0.7284;
    - DummyClassifier - 0.1755.
 

2. Значение F1-меры обратно коррелирует со средним между ложно-положительными и ложно-отрицательными ответами, что, собственно, следует из определения данной метрики. Т.е. чем меньше суммарно ложных ответов, тем выше значение F1. Однако минимизация прежде всего ложно-отрицательных ответов должна быть критерием выбора оптимальной модели, поскольку этот элемент матрицы ошибок показывает количество пропущенных токсичных комментариев. В отличие от него, ложно-положительные ответы - это всего лишь не прошедшие модерацию нормальные комментарии, которые пользователь всегда может переслать/переписать. Иными словами, полнота (Recall), которая как раз обратно коррелирует с ложно-отрицательными ответами, имеет также важное значение.


3. По F1-мере наиболее высокое качество у модели model_lgbmc(LGBMClassifier) - 0.7691 (значение полноты (recall) - 0.7904). По полноте же лучшие показатели у моделей model_rfc (RandomForestClassifier) - 0.8795 и model_lr (LogisticRegression) - 0.8251. Проверку на вменяемость через DummyClassifier модель model_lr также прошла успешо. Цель исследования достигнута, требование "F1 >= 0.75" выполнено.

## Общие выводы

1. __Задача исследования.__ Для нового сервиса Интернет-магазина, дающего пользователям возможность редактировать и дополнять описания товаров, предлагая свои правки и комментируя изменения других, необходимо разработать модель машинного обучения, способную искать токсичные комментарии и отправлять их на модерацию. Модель должна быть обучена классифицировать комментарии на токсичные и нетоксичные, со значением метрики качества F1 не меньше 0.75.  Для исследования предоставлен набор данных с разметкой о токсичности правок.


2. __Изучение данных__. Таблица представляет собой датасет с 159292 строками и 2 столбцами. В столбце 'text' содержатся тексты сообщений, тип данных - object. В столбце 'toxic' - целевой признак в виде 1 и 0, тип данных - int64. Пропущенных значений в таблице нет. В целевом признаке наблюдается дисбаланс - почти 9:1 в пользу отрицательного класса (доля нулей - 89.8%, доля единиц - 10.2%). Полных дубликатов стреди строк не обнаружено. Однако следует перепроверить данный факт после очищения текстов. 


3. __Обработка данных__. Для сокращения времени на обработку, а также восстановления баланса классов в целевом признаке выполнен даунсемплинг до соответствующих размеров: количество строк с нулями в целевом признаке уменьшен в кратное количество раз, до уровня положительного класса. Общее количество строк уменьшилось до 32372, т.е. примерно в 5 раз. Тексты комментариев переведены в нижний регистр. Проведено очищение текстов от лишних символов, кроме латинских букв и апострофа. Также проведена токенизация текстов и лемматизация слов в них. Обнаружены и удалены дублирующиеся строки. Из датафрейма выделен признак для обучения ('text') и целевой признак ('toxic'), с переводом в векторный вид. Каждый датасет разбит на обучающую и тестовую выборки. Сформирован набор стоп-слов и создан счетчик величин TF-IDF. Создан корпус текстов в обучающей выборке, а также созданы признаки (в обучающей и тестовой выборках) из величин TF-IDF.


4. __Подбор параметров__. На обучающей выборке проверено пять структур машинного обучения: Логистрическая регрессия, Классификатор Ridge, Классификатор случайного леса, Классификатор LGBM и Классификатор CatBoost, а также фиктивный Классификатор Dummy. Для каждой структуры подобраны лучшие гиперпараметры. По итогам подбора, меньше всего времени на обработку ушло у модели RidgeClassifier (1 минута), больше всего - у LGBMClassifier (около 30 минут). Наиболее высокий показатель F1-меры на обучающей выборке продемонстрировала модель LogisticRegression (0.8545), наиболее низкий - RandomForestClassifier (0.7012).


5. __Тестирование моделей__. Обученные на подобранных гиперпараметрах модели показали на тестовой выборке следующие значения метрики F1 : LogisticRegression - 0.7559; RidgeClassifier - 0.7279; RandomForestClassifier - 0.3694; LGBMClassifier - 0.7691; CatBoostClassifier - 0.7284; DummyClassifier - 0.1755. Не только по значению F1-меры, но также еще по двум метрикам: правильности (accuracy) - 0.9514 и точности (precision) - 0.7488, - лучшие показатели у модели model_lgbmc (LGBMClassifier). Значение полноты (recall) также на довольно высоком уровне у этой модели - 0.7904. Однако лучшие показатели по метрике Recall у моделей model_rfc (RandomForestClassifier) - 0.8795 и model_lr (LogisticRegression) - 0.8251. Но, учитывая, что у классификатора случайного леса довольно низкий уровень метрики F1, ее в качестве модели машинного обучения мы не можем рассматривать. В итоге лучшими оказались моделями классификатора LGBM и логистической регрессии, но у первой значение F1-меры на 0.013 выше. Проверку на вменяемость через DummyClassifier модель model_lgbmc также прошла успешо. Цель исследования достигнута, требование "F1 >= 0.75" выполнено: значение метрики F1 - 0.7691.


6. __Дополнительные выводы__. Поиск модели, дающей максимальное значение F1-меры, имеет важный смысл. Сама данная метрика складывается из сочетания точности и полноты, а именно является их средней гармонической. Точность равна отношению истинно-положительных ответов к их сумме с ложно-положительными (TP/(TP+FP)). Полнота же равна отношению истинно-положительных ответов к их сумме с ложно-отрицательными (TP/(TP+FN)). Т.е. их отличие только в одном из слагаемых в знаменателе (FP или FN). Если на место этих двух разных слагаемых вписать их среднее арифметическое ((FP+FN)/2, или просто F/2), то мы как раз получим F1-меру. Таким образом, эта метрика придает одинаковую важность точности и полноте, обратно коррелируя со средним между ложно-положительными и ложно-отрицательными ответами. Однако считаю необходимым добавить, что именно количество ложно-отрицательных ответов показывает, сколько было пропущено токсичных комментариев. В отличие от него, ложно-положительные ответы - это всего лишь не прошедшие модерацию нормальные комментарии, которые пользователь всегда может переписать. Т.е. полнота (Recall), которая как раз обратно коррелирует с ложно-отрицательными ответами, имеет не менее важное значение для целей настоящего исследования.